In [1]:
#import all the required libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import the train file only
df_train=pd.read_csv('train.csv')
df_center_info = pd.read_csv('fulfilment_center_info.csv')
df_meal_info = pd.read_csv('meal_info.csv')

In [3]:
df_test=pd.read_csv('test_food.csv')

In [4]:
df_train.head()

id  week  center_id  meal_id  checkout_price  base_price  emailer_for_promotion  homepage_featured  num_orders
0  1379560     1         55     1885          136.83      152.29                      0                  0         177
1  1466964     1         55     1993          136.83      135.83                      0                  0         270
2  1346989     1         55     2539          134.86      135.86                      0                  0         189
3  1338232     1         55     2139          339.50      437.53                      0                  0          54
4  1448490     1         55     2631          243.50      242.50                      0                  0          40

In [ ]:
df_train.dtypes

In [5]:
df_test.head()

id  week  center_id  meal_id  checkout_price  base_price  emailer_for_promotion  homepage_featured
0  1028232   146         55     1885          158.11      159.11                      0                  0
1  1127204   146         55     1993          160.11      159.11                      0                  0
2  1212707   146         55     2539          157.14      159.14                      0                  0
3  1082698   146         55     2631          162.02      162.02                      0                  0
4  1400926   146         55     1248          163.93      163.93                      0                  0

In [6]:
df_center_info.head()

center_id  city_code  region_code center_type  op_area
0         11        679           56      TYPE_A      3.7
1         13        590           56      TYPE_B      6.7
2        124        590           56      TYPE_C      4.0
3         66        648           34      TYPE_A      4.1
4         94        632           34      TYPE_C      3.6

In [7]:
df_meal_info.head()

meal_id   category cuisine
0     1885  Beverages    Thai
1     1993  Beverages    Thai
2     2539  Beverages    Thai
3     1248  Beverages  Indian
4     2631  Beverages  Indian

In [8]:
#merge the train table with the center_info table on center_id
df_train=pd.merge(df_train,df_center_info,how='left',left_on='center_id',right_on='center_id')

In [9]:
#merge the train table with the meal_info table on meal_id
df_train=pd.merge(df_train,df_meal_info,how='left',left_on='meal_id',right_on='meal_id')

In [10]:
#merge the test table with the center_info table on center_id
df_test=pd.merge(df_test,df_center_info,how='left',left_on='center_id',right_on='center_id')

In [11]:
#merge the test table with the meal_info table on meal_id
df_test=pd.merge(df_test,df_meal_info,how='left',left_on='meal_id',right_on='meal_id')

In [12]:
df_train.head()

id  week  center_id  meal_id  checkout_price  base_price  emailer_for_promotion  homepage_featured  num_orders  city_code  region_code center_type  op_area   category cuisine
0  1379560     1         55     1885          136.83      152.29                      0                  0         177        647           56      TYPE_C      2.0  Beverages    Thai
1  1466964     1         55     1993          136.83      135.83                      0                  0         270        647           56      TYPE_C      2.0  Beverages    Thai
2  1346989     1         55     2539          134.86      135.86                      0                  0         189        647           56      TYPE_C      2.0  Beverages    Thai
3  1338232     1         55     2139          339.50      437.53                      0                  0          54        647           56      TYPE_C      2.0  Beverages  Indian
4  1448490     1         55     2631          243.50      242.50                      0                  0          40        647           56      TYPE_C      2.0  Beverages  Indian

In [13]:
df_test.head()

id  week  center_id  meal_id  checkout_price  base_price  emailer_for_promotion  homepage_featured  city_code  region_code center_type  op_area   category cuisine
0  1028232   146         55     1885          158.11      159.11                      0                  0        647           56      TYPE_C      2.0  Beverages    Thai
1  1127204   146         55     1993          160.11      159.11                      0                  0        647           56      TYPE_C      2.0  Beverages    Thai
2  1212707   146         55     2539          157.14      159.14                      0                  0        647           56      TYPE_C      2.0  Beverages    Thai
3  1082698   146         55     2631          162.02      162.02                      0                  0        647           56      TYPE_C      2.0  Beverages  Indian
4  1400926   146         55     1248          163.93      163.93                      0                  0        647           56      TYPE_C      2.0  Beverages  Indian

In [14]:
# Convert 'city_code' and 'region_code' into a single feature - 'city_region'.
df_train['city_region'] = \
        df_train['city_code'].astype('str') + '_' + \
        df_train['region_code'].astype('str')

In [15]:
df_test['city_region'] = \
        df_test['city_code'].astype('str') + '_' + \
        df_test['region_code'].astype('str')

In [16]:
#Label encode categorical columns for use in regression
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
label_encode_cols=['center_id','meal_id','city_code','region_code','city_region','center_type','category','cuisine']

In [17]:
for col in label_encode_cols:
    le.fit(df_train[col])
    df_train[col+'_encoded']=le.transform(df_train[col])
    df_test[col + '_encoded'] = le.transform(df_test[col])

In [18]:
# Feature engineering - treat 'week' as a cyclic feature.
# Encode it using sine and cosine transform.
df_train['week_sin'] = \
        np.sin(2 * np.pi * df_train['week'] / 52.143)
df_train['week_cos'] = \
        np.cos(2 * np.pi * df_train['week'] / 52.143)
df_test['week_sin'] = \
        np.sin(2 * np.pi * df_test['week'] / 52.143)
df_test['week_cos'] = \
        np.cos(2 * np.pi * df_test['week'] / 52.143)

In [19]:
# Feature engineering - percent difference between base price and checkout price.
df_train['price_diff_percent'] = \
        (df_train['base_price'] - df_train['checkout_price']) / \
        df_train['base_price']
df_test['price_diff_percent'] = \
        (df_test['base_price'] - df_test['checkout_price']) / \
        df_test['base_price']

In [20]:
# Convert email and homepage features into a single feature - 'email_plus_homepage'.
df_train['email_plus_homepage'] = \
        df_train['emailer_for_promotion'] + \
        df_train['homepage_featured']
df_test['email_plus_homepage'] = \
        df_test['emailer_for_promotion'] + \
        df_test['homepage_featured']

In [21]:
df_train.head(2)

id  week  center_id  meal_id  checkout_price  base_price  emailer_for_promotion  homepage_featured  num_orders  city_code  region_code center_type  op_area   category cuisine city_region  center_id_encoded  meal_id_encoded  city_code_encoded  region_code_encoded  city_region_encoded  center_type_encoded  category_encoded  cuisine_encoded  week_sin  week_cos  price_diff_percent  email_plus_homepage
0  1379560     1         55     1885          136.83      152.29                      0                  0         177        647           56      TYPE_C      2.0  Beverages    Thai      647_56                 23               22                 30                    3                   30                    2                 0                3  0.120208  0.992749            0.101517                    0
1  1466964     1         55     1993          136.83      135.83                      0                  0         270        647           56      TYPE_C      2.0  Beverages    Thai      647_56                 23               26                 30                    3                   30                    2                 0                3  0.120208  0.992749           -0.007362                    0

In [22]:
# Prepare a list of columns to train on.
# Also decide which features to treat as numeric and which features to treat
# as categorical.
columns_to_train=['week',
                  'week_sin',
                  'week_cos',
                  'cuisine_encoded',
                  'category_encoded',
                  'center_type_encoded',
                  'email_plus_homepage',
                  'price_diff_percent',
                  'city_region_encoded',
                  'meal_id_encoded',
                  'center_id_encoded',
                  'op_area',
                  'base_price',
                  'checkout_price']

In [23]:
#list of categorically encoded columns
categorical_columns=['email_plus_homepage',
                       'city_region_encoded',
                       'center_type_encoded',
                       'category_encoded',
                       'cuisine_encoded',
                       'center_id_encoded',
                       'meal_id_encoded']

In [24]:
#list of numerical columns
numerical_columns=[col for col in columns_to_train if col not in categorical_columns ]

In [25]:
MODELS = []
MAE = []
MSE = []
RMSE = []
RMSLE = []
DIFFERENCE = []

In [26]:
# Log transform the target variable - num_orders.
df_train['num_orders_log1p'] = np.log1p(df_train['num_orders'])

In [27]:
# Train-Test split.
from sklearn.model_selection import train_test_split
X = df_train[categorical_columns + numerical_columns]
y = df_train['num_orders_log1p']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    shuffle=False)

In [28]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [29]:
X_train = sc.fit_transform(X_train)

In [30]:
sc.mean_

array([1.92200702e-01, 2.26567060e+01, 6.42060793e-01, 5.26804714e+00,
       1.56784343e+00, 3.74834683e+01, 2.52748591e+01, 6.05624552e+01,
       6.55030378e-02, 6.83393192e-02, 5.72181674e-02, 4.08284461e+00,
       3.52072023e+02, 3.30471672e+02])

In [31]:
sc.var_

array([2.38290383e-01, 1.90818815e+02, 6.66596941e-01, 2.00286852e+01,
       1.21149609e+00, 5.01084881e+02, 2.17464147e+02, 1.13020230e+03,
       4.98486621e-01, 4.92552469e-01, 1.30590854e-02, 1.18965055e+00,
       2.55574974e+04, 2.33694104e+04])

In [32]:
X_test = sc.transform(X_test)

In [33]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet,LassoLars,BayesianRidge,SGDRegressor,PassiveAggressiveRegressor,HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor

In [34]:
lr=LinearRegression()
lasso=Lasso()
rd=Ridge()
els=ElasticNet()
lar=LassoLars()
br=BayesianRidge()
sgd=SGDRegressor()
par=PassiveAggressiveRegressor()
hub=HuberRegressor()
dtr=DecisionTreeRegressor()
rfr=RandomForestRegressor()
adb=AdaBoostRegressor()

In [35]:
lr.fit(X_train,y_train)
MODELS.append('Linear Regression')
y_pred_lr=lr.predict(X_test)
lasso.fit(X_train,y_train)
MODELS.append('Lasso Regression')
y_pred_lasso=lasso.predict(X_test)
rd.fit(X_train,y_train)
MODELS.append('Ridge Regression')
y_pred_rd=rd.predict(X_test)
els.fit(X_train,y_train)
MODELS.append('Elastic Net Regression')
y_pred_els=els.predict(X_test)
lar.fit(X_train,y_train)
MODELS.append('LasoLArs Regression')
y_pred_lar=lar.predict(X_test)
br.fit(X_train,y_train)
MODELS.append('Bayesian Ridge Regression')
y_pred_br=br.predict(X_test)
sgd.fit(X_train,y_train)
MODELS.append('SGD Regression')
y_pred_sgd=sgd.predict(X_test)
par.fit(X_train,y_train)
MODELS.append('PassiveAgressive Regression')
y_pred_par=par.predict(X_test)
hub.fit(X_train,y_train)
MODELS.append('Huber Regression')
y_pred_hub=hub.predict(X_test)
dtr.fit(X_train,y_train)
MODELS.append('DecisionTree Regression')
y_pred_dtr=dtr.predict(X_test)
rfr.fit(X_train,y_train)
MODELS.append('Random Forest Regression')
y_pred_rfr=rfr.predict(X_test)
adb.fit(X_train,y_train)
MODELS.append('AdaBoost Regression')
y_pred_adb=adb.predict(X_test)

In [36]:
#Average Ensemble
# Fitting Voting Regressor to the Training set
from sklearn.ensemble import VotingRegressor
vc2 = VotingRegressor(estimators=[('Linear Regression',lr),
                                   ('Lasso Regression',lasso),
                                   ('Ridge Regression',rd),
                                   ('Elastic Net Regression',els),
                                   ('LasoLArs Regression',lar),
                                ('Bayesian Ridge Regression',br),
                                  ('SGD Regression',sgd),
                                  ('PassiveAgressive Regression',par),
                                  ('Huber Regression',hub),
                                 ('Decision Tree',dtr),
                                 ('Random Forest',rfr),
                                 ('AdaBoost',adb)])
vc2.fit(X_train, y_train)
MODELS.append('Average Ensemble')

In [37]:
y_pred_esemble= vc2.predict(X_test)
y_pred_esemble

array([5.28549069, 6.46588081, 5.17749772, ..., 5.26898345, 5.32126001,
       5.08722387])

In [38]:
MODELS

['Linear Regression',
 'Lasso Regression',
 'Ridge Regression',
 'Elastic Net Regression',
 'LasoLArs Regression',
 'Bayesian Ridge Regression',
 'SGD Regression',
 'PassiveAgressive Regression',
 'Huber Regression',
 'DecisionTree Regression',
 'Random Forest Regression',
 'AdaBoost Regression',
 'Average Ensemble']

In [39]:
#Evaluation through metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_squared_log_error

In [40]:
#Linear_Regression
mse = mean_squared_error(y_test,y_pred_lr)
mae= mean_absolute_error(y_test,y_pred_lr)
rmsle= mean_squared_log_error(y_test,y_pred_lr)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.9466571546576767
RMSE:  0.9729630798019403
MAE:  0.7943936897750727
RMSLE:  0.030544473672910653
Difference 0.17856939002686756


In [41]:
#Lasso Regression
mse = mean_squared_error(y_test,y_pred_lasso)
mae= mean_absolute_error(y_test,y_pred_lasso)
rmsle= mean_squared_log_error(y_test,y_pred_lasso)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  1.3903626650577428
RMSE:  1.179136406467777
MAE:  0.9689146687599965
RMSLE:  0.0449187004693104
Difference 0.2102217377077804


In [42]:
#Ridge Regression
mse = mean_squared_error(y_test,y_pred_rd)
mae= mean_absolute_error(y_test,y_pred_rd)
rmsle= mean_squared_log_error(y_test,y_pred_rd)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.9466587751887959
RMSE:  0.9729639125829878
MAE:  0.7943949363123933
RMSLE:  0.03054453224435614
Difference 0.17856897627059443


In [43]:
#Elastic-Net Regression
mse = mean_squared_error(y_test,y_pred_els)
mae= mean_absolute_error(y_test,y_pred_els)
rmsle= mean_squared_log_error(y_test,y_pred_els)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  1.3903626650577428
RMSE:  1.179136406467777
MAE:  0.9689146687599965
RMSLE:  0.0449187004693104
Difference 0.2102217377077804


In [44]:
#LassoLars Regression
mse = mean_squared_error(y_test,y_pred_lar)
mae= mean_absolute_error(y_test,y_pred_lar)
rmsle= mean_squared_log_error(y_test,y_pred_lar)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  1.3903626650577428
RMSE:  1.179136406467777
MAE:  0.9689146687599965
RMSLE:  0.0449187004693104
Difference 0.2102217377077804


In [45]:
#Bayesian Ridge Regression
mse = mean_squared_error(y_test,y_pred_br)
mae= mean_absolute_error(y_test,y_pred_br)
rmsle= mean_squared_log_error(y_test,y_pred_br)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.9466718625305637
RMSE:  0.9729706380618912
MAE:  0.7944049931667185
RMSLE:  0.030545004993208752
Difference 0.17856564489517268


In [46]:
#SGD Regression
mse = mean_squared_error(y_test,y_pred_sgd)
mae= mean_absolute_error(y_test,y_pred_sgd)
rmsle= mean_squared_log_error(y_test,y_pred_sgd)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.9467451074236058
RMSE:  0.9730082771608913
MAE:  0.7948114677133992
RMSLE:  0.0305737240754564
Difference 0.17819680944749217


In [47]:
#PassiveAgressive Regression
mse = mean_squared_error(y_test,y_pred_par)
mae= mean_absolute_error(y_test,y_pred_par)
rmsle= mean_squared_log_error(y_test,y_pred_par)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  2.283140058488107
RMSE:  1.5110063065679464
MAE:  1.1988512655010182
RMSLE:  0.06543517329345502
Difference 0.31215504106692826


In [48]:
#Huber Regression
mse = mean_squared_error(y_test,y_pred_hub)
mae= mean_absolute_error(y_test,y_pred_hub)
rmsle= mean_squared_log_error(y_test,y_pred_hub)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.9515041910258343
RMSE:  0.975450762994132
MAE:  0.7881681314474971
RMSLE:  0.0308881993285794
Difference 0.18728263154663494


In [49]:
#Average Ensemble 
mse = mean_squared_error(y_test,y_pred_esemble)
mae= mean_absolute_error(y_test,y_pred_esemble)
rmsle= mean_squared_log_error(y_test,y_pred_esemble)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.7655499147060013
RMSE:  0.8749570930657121
MAE:  0.7196211991032926
RMSLE:  0.025201148826089752
Difference 0.15533589396241954


In [50]:
#Decision Tree
mse = mean_squared_error(y_test,y_pred_dtr)
mae= mean_absolute_error(y_test,y_pred_dtr)
rmsle= mean_squared_log_error(y_test,y_pred_dtr)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.6791571347164654
RMSE:  0.8241099045130239
MAE:  0.617629064051088
RMSLE:  0.024922164302698096
Difference 0.20648084046193593


In [51]:
#RandomForest
mse = mean_squared_error(y_test,y_pred_rfr)
mae= mean_absolute_error(y_test,y_pred_rfr)
rmsle= mean_squared_log_error(y_test,y_pred_rfr)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.38514987835310316
RMSE:  0.6206044459662717
MAE:  0.47655158266743525
RMSLE:  0.013508989343962366
Difference 0.1440528632988365


In [52]:
#AdaBoost
mse = mean_squared_error(y_test,y_pred_adb)
mae= mean_absolute_error(y_test,y_pred_adb)
rmsle= mean_squared_log_error(y_test,y_pred_adb)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)
print('MAE: ',mae)
print('RMSLE: ',rmsle)
diff=abs(mae-rmse)
print('Difference',diff)
MAE.append(mae)
MSE.append(mse)
RMSE.append(rmse)
RMSLE.append(rmsle)
DIFFERENCE.append(diff)

MSE:  0.7704745122822652
RMSE:  0.8777667755629995
MAE:  0.7217807074323412
RMSLE:  0.024820908811955776
Difference 0.1559860681306583


In [53]:
#Convert the metrics values into a data table
model_dict = {'Models': MODELS,
             'Mean Squared Error': MSE,
             'Mean Absolute Error': MAE,
             'Root Mean Squared Error': RMSE,
             'Root Mean Squared Log Error': RMSLE,
             'Difference': DIFFERENCE}

In [54]:
model_df = pd.DataFrame(model_dict)
model_df

Models  Mean Squared Error  Mean Absolute Error  Root Mean Squared Error  Root Mean Squared Log Error  Difference
0             Linear Regression            0.946657             0.794394                 0.972963                     0.030544    0.178569
1              Lasso Regression            1.390363             0.968915                 1.179136                     0.044919    0.210222
2              Ridge Regression            0.946659             0.794395                 0.972964                     0.030545    0.178569
3        Elastic Net Regression            1.390363             0.968915                 1.179136                     0.044919    0.210222
4           LasoLArs Regression            1.390363             0.968915                 1.179136                     0.044919    0.210222
5     Bayesian Ridge Regression            0.946672             0.794405                 0.972971                     0.030545    0.178566
6                SGD Regression            0.946745             0.794811                 0.973008                     0.030574    0.178197
7   PassiveAgressive Regression            2.283140             1.198851                 1.511006                     0.065435    0.312155
8              Huber Regression            0.951504             0.788168                 0.975451                     0.030888    0.187283
9       DecisionTree Regression            0.765550             0.719621                 0.874957                     0.025201    0.155336
10     Random Forest Regression            0.679157             0.617629                 0.824110                     0.024922    0.206481
11          AdaBoost Regression            0.385150             0.476552                 0.620604                     0.013509    0.144053
12             Average Ensemble            0.770475             0.721781                 0.877767                     0.024821    0.155986

In [55]:
model_df = model_df.sort_values(['Mean Absolute Error','Root Mean Squared Error','Root Mean Squared Log Error','Difference'],
                               ascending=True)

In [56]:
model_df

Models  Mean Squared Error  Mean Absolute Error  Root Mean Squared Error  Root Mean Squared Log Error  Difference
11          AdaBoost Regression            0.385150             0.476552                 0.620604                     0.013509    0.144053
10     Random Forest Regression            0.679157             0.617629                 0.824110                     0.024922    0.206481
9       DecisionTree Regression            0.765550             0.719621                 0.874957                     0.025201    0.155336
12             Average Ensemble            0.770475             0.721781                 0.877767                     0.024821    0.155986
8              Huber Regression            0.951504             0.788168                 0.975451                     0.030888    0.187283
0             Linear Regression            0.946657             0.794394                 0.972963                     0.030544    0.178569
2              Ridge Regression            0.946659             0.794395                 0.972964                     0.030545    0.178569
5     Bayesian Ridge Regression            0.946672             0.794405                 0.972971                     0.030545    0.178566
6                SGD Regression            0.946745             0.794811                 0.973008                     0.030574    0.178197
1              Lasso Regression            1.390363             0.968915                 1.179136                     0.044919    0.210222
3        Elastic Net Regression            1.390363             0.968915                 1.179136                     0.044919    0.210222
4           LasoLArs Regression            1.390363             0.968915                 1.179136                     0.044919    0.210222
7   PassiveAgressive Regression            2.283140             1.198851                 1.511006                     0.065435    0.312155

In [57]:
best_model = model_df['Models'].values[0]
best_model

'AdaBoost Regression'

In [58]:
# Get predictions on the test set and prepare submission file.
X = df_test[categorical_columns + numerical_columns]

pred = adb.predict(X)
pred = np.expm1(pred)

submission_df = df_test.copy()
submission_df['num_orders'] = pred
submission_df = submission_df[['id', 'num_orders']]
submission_df.to_csv('submission.csv', index=False)